In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# # delete this cell if working on Pycharm
!pip install Bio
!pip install import-ipynb

In [3]:
%cd "/content/drive/MyDrive/bio_ex3_new/"

/content/drive/MyDrive/bio_ex3_new


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# so we can import utils notebook (delete if working on Pycharm), you might need to change it to your working directory path

import import_ipynb
import utils

In [5]:
###############################################################################
#                                                                             #
#              Parameters you can change, but don't have to                   #
#                                                                             #
###############################################################################


# number of ResNet blocks for the first ResNet and the kernel size.
RESNET_1_BLOCKS = 3
RESNET_1_KERNEL_SIZE = 15
RESNET_1_KERNEL_NUM = 64


###############################################################################
#                                                                             #
#                        Parameters you need to choose                        #
#                                                                             #
###############################################################################


# number of ResNet blocks for the second ResNet, dilation list to repeat and the kernel size.

RESNET_2_BLOCKS = 3
RESNET_2_KERNEL_SIZE = 3  # good start may be 3/5
RESNET_2_KERNEL_NUM = 128
DILATION = [1,2,4,8]

# percentage of dropout for the dropout layer
DROPOUT = 0.25 # good start may be 0.1-0.5|

# number of epochs, Learning rate and Batch size
EPOCHS = 20
LR = 0.001 # good start may be 0.0001/0.001/0.01
BATCH = 64 # good start may be 32/64/128

In [6]:
def resnet_1(input_layer):
    """
    ResNet layer - input -> BatchNormalization -> Conv1D -> Relu -> BatchNormalization -> Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for i in range(RESNET_1_BLOCKS):
        batch_norm_layer = layers.BatchNormalization()(input_layer)
        conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, activation='relu', padding='same')(batch_norm_layer)
        batch_norm_layer = layers.BatchNormalization()(conv1d_layer)
        input_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, activation='relu', padding='same')(batch_norm_layer) + input_layer
    return input_layer


In [7]:
def resnet_2(input_layer):  # TODO: implement this!
    """
    Dilated ResNet layer - input -> BatchNormalization -> dilated Conv1D -> Relu -> BatchNormalization -> dilated Conv1D -> Relu -> Add
    :param input_layer: input layer for the ResNet
    :return: last layer of the ResNet
    """
    for i in range(RESNET_2_BLOCKS):
        for j in range(len(DILATION)):
            batch_norm_layer = layers.BatchNormalization()(input_layer)
            conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, dilation_rate=DILATION[j], activation='relu', padding='same')(batch_norm_layer)
            batch_norm_layer = layers.BatchNormalization()(conv1d_layer)
            input_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, dilation_rate=DILATION[j], activation='relu', padding='same')(batch_norm_layer) + input_layer
    return input_layer

In [8]:
def build_network():
    """
    builds the neural network architecture as shown in the exercise.
    :return: a Keras Model
    """
    # input, shape (NB_MAX_LENGTH,FEATURE_NUM)
    input_layer = tf.keras.Input(shape=(utils.NB_MAX_LENGTH, utils.FEATURE_NUM))

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_1_KERNEL_NUM, RESNET_1_KERNEL_SIZE, padding='same')(input_layer)

    # first ResNet -> shape = (NB_MAX_LENGTH, RESNET_1_KERNEL_NUM)
    resnet_layer = resnet_1(conv1d_layer)

    # Conv1D -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM, RESNET_2_KERNEL_SIZE, padding="same")(resnet_layer)

    # second ResNet -> shape = (NB_MAX_LENGTH, RESNET_2_KERNEL_NUM)
    resnet_layer = resnet_2(conv1d_layer)

    dropout_layer = layers.Dropout(DROPOUT)(resnet_layer)

    conv1d_layer = layers.Conv1D(RESNET_2_KERNEL_NUM // 2, RESNET_2_KERNEL_SIZE, padding="same")(dropout_layer)

    elu_layer = layers.ELU()(conv1d_layer)

    output_layer = layers.Dense(utils.OUTPUT_SIZE)(elu_layer)
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model





In [9]:
def plot_val_train_loss(history):
    """
    plots the train and validation loss of the model at each epoch, saves it in 'model_loss_history.png'
    :param history: history object (output of fit function)
    :return: None
    """
    ig, axes = plt.subplots(1, 1, figsize=(15,3))
    axes.plot(history.history['loss'], label='Training loss')
    axes.plot(history.history['val_loss'], label='Validation loss')
    axes.legend()
    axes.set_title("Train and Val MSE loss")

    plt.savefig("/content/drive/MyDrive/bio_ex3_new/model_loss_history")  # TODO: you can change the path here


In [10]:
!ls

6xw6	 model.keras		 net.ipynb  train_input.npy   utils.ipynb
Ex4Data  model_loss_history.png  README.md  train_labels.npy


In [ ]:
if __name__ == '__main__':

    model = build_network()

    # you can load here your input and output data

    X = np.load("/content/drive/MyDrive/bio_ex3_new/train_input.npy")
    Y = np.load("/content/drive/MyDrive/bio_ex3_new/train_labels.npy")

    # X = numpy array of shape (1974,NB_MAX_LENGTH,FEATURE_NUM) of all the data input.
    # Y = numpy array of shape (1974,NB_MAX_LENGTH,OUTPUT_SIZE) of all the data labels.

    # split into validation and test sets as you like

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

    # b)
    #  compile model using Adam optimizer (with learning rate of your choice) and MSE loss.
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR), loss='mse')

    # c)
    # fit model (use EPOCH for epoch parameter and BATCH for batch_size parameter)
    history = model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH, validation_data=(X_test, Y_test))
    plot_val_train_loss(history)

    # d)
    # save model
    model.save("/content/drive/MyDrive/bio_ex3_new/model.keras")



In [ ]:
print(model.summary())

In [11]:
#load model
model = tf.keras.models.load_model("/content/drive/MyDrive/bio_ex3_new/model.keras", compile=False)


In [12]:
# get seq
seq = []
with open("6xw6/6xw6.fasta") as fh:
    for line in fh:
        if line.startswith(">"):
            continue
        seq.append(line.strip())

sequence_str = "".join(seq)
print(sequence_str)

VQLQESGGGLVQAGDSLRVSCAASGRTISSSPMGWFRQAPGKEREFVAAISGNGGNTYYLDSVKGRFTTSRDNAKNTVYLQLNNLKPEDTAIYYCAARSRFSAMHLAYRRLVDYDDWGQGTQVTVS


In [13]:
# predict
data_path = '/content/drive/MyDrive/bio_ex3_new/6xw6/6xw6.pdb'
input_one_hot = utils.generate_input(data_path)
input_one_hot = np.expand_dims(input_one_hot, axis=0)
prediction = model.predict(input_one_hot)
prediction = np.squeeze(prediction)
output_pdb_file_name = '6xw6_predicted'
utils.matrix_to_pd(sequence_str, prediction, output_pdb_file_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
